In [22]:
import torch
import torch.nn as nn
from data_rnn import load_ndfa, load_brackets
from utils import device

In [6]:
x_train, (i2w, w2i) = load_ndfa(n=150_000)

In [15]:
''.join([i2w[i] for i in x_train[-1]])

'sabc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!abc!s'

In [53]:
class Network(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.emb = nn.Embedding(len(i2w), 32)
        self.lstm = nn.LSTM(32, 16, batch_first=True)
        self.linear = nn.Linear(16, len(w2i))
    
    def forward(self, x):
        x = self.emb(x)
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x

In [54]:
model = Network().to(device)

In [56]:
model(torch.tensor(x_train[0], dtype=torch.long, device=device).unsqueeze(dim=0)).shape

torch.Size([1, 2, 15])

In [5]:
x_train, (i2w, w2i) = load_brackets(n=150_000)